In [4]:
import json
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import sys
sys.path.append('../../assets/python/')
import estats5e
import tfb

METADATA = {'Contributor': 'T. Dunn'}
SAVEFIGS = False

PC_CLASSES = {
    'Barbarian': {'color': '#E7623E', 'group': 'martial'},
    'Bard': {'color': '#AB6DAC', 'group': 'spellcaster'},
    'Cleric': {'color': '#91A1B2', 'group': 'spellcaster'},
    'Druid': {'color': '#7A853B', 'group': 'spellcaster'},
    'Fighter': {'color': '#7F513E', 'group': 'martial'},
    'Monk': {'color': '#51A5C5', 'group': 'martial'},
    'Paladin': {'color': '#B59E54', 'group': 'martial-spellcaster'},
    'Ranger': {'color': '#507F62', 'group': 'martial-spellcaster'},
    'Rogue': {'color': '#555752', 'group': 'martial'},
    'Sorcerer': {'color': '#992E2E', 'group': 'spellcaster'},
    'Warlock': {'color': '#7B469B', 'group': 'spellcaster'},
    'Wizard': {'color': '#2A50A1', 'group': 'spellcaster'}
}

ENCOUNTER_DIFFICULTIES = {
    'Easy':   {
        'XP': [25,50,75,125,250,300,350,450,550,600,800,1000,1100,1250,1400,1600,2000,2100,2400,2800], 
        'color': '#1F77B4'
    },
    'Medium': {
        'XP': [50,100,150,250,500,600,750,900,1100,1200,1600,2000,2200,2500,2800,3200,3900,4200,4900,5700], 
        'color': '#FF7F0E'
    },
    'Hard':   {
        'XP': [75,150,225,375,750,900,1100,1400,1600,1900,2400,3000,3400,3800,4300,4800,5900,6300,7300,8500], 
        'color': '#2CA02C'
    },
    'Deadly': {
        'XP': [100,200,400,500,1100,1400,1700,2100,2400,2800,3600,4500,5100,5700,6400,7200,8800,9500,10900,12700], 
        'color': '#D62728'
    },
    'Daily':  {
        'XP': [300,600,1200,1700,3500,4000,5000,6000,7500,9000,10500,11500,13500,15000,18000,20000,25000,27000,30000,40000], 
        'color': '#9467BD'
    },
}

In [5]:
# import data
def legendary_resistance_HP(cr):
    if cr <= 4:
        return 10
    elif cr <= 10:
        return 20
    else:
        return 30

dfD = pd.read_csv('../../assets/data/dmg-targets.csv') # 'CR','PB','XP','AC','HP Mean','AB','DPR Mean'
dfD['eHP Mean']  = dfD.apply(lambda row: estats5e.effHP(row['HP Mean'], row['AC']), axis=1)
dfD['eDPR Mean'] = dfD.apply(lambda row: estats5e.effDPR(row['DPR Mean'], row['AB']), axis=1)
dfD['eXP Mean']  = dfD.apply(lambda row: estats5e.effXP(row['HP Mean'], row['AC'], row['DPR Mean'], row['AB']), axis=1)
dfD['Legendary Resistance HP'] = dfD.apply(lambda row: legendary_resistance_HP(row['CR']), axis=1)
dfD['Legendary Resistance eHP']  = dfD.apply(lambda row: estats5e.effHP(row['Legendary Resistance HP'], row['AC']), axis=1)

In [6]:
# Plots eDPR for each class, calculated for a full adventuring day with Medium encounters and two short rests.
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        xaxis=dict(
            title_text='monster CR (PC level)',
            range=[0.8, 20.2],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='monster effective HP (PC effective DPR)',
            range=[0, 90],
            tick0=0, dtick=10,
            minor=dict(tick0=0, dtick=5),
        ),
        legend=dict(
            yanchor='top',  y=0.99,
            xanchor='left', x=0.01,
        )
    )
)

encounter_difficulty = 'Medium'
pc_data_file = f'../../_classes/xp-and-player-characters/pc data - {encounter_difficulty} 2 short rests.json'

with open(pc_data_file, 'r') as fin:
    pc_data = json.load(fin)

levels = np.array(range(1,21,1))

eDPRmean = np.zeros(20)
for pc in PC_CLASSES:
    eDPRmean += np.array([pc_data[pc][lvl]['effective damage per round mean'] for lvl in range(20)])

eDPRmean /= len(PC_CLASSES)
fig.add_trace(go.Scatter(
    x=levels, 
    y=eDPRmean,
    mode='lines+markers', 
    line=dict(color='black', dash='dash'),
    unselected=dict(marker=dict(opacity=0.0)),
    name='PC average effective DPR',
    hovertemplate = '<b>PC Level</b> %{x}<br>'
        + '<b>eDPR</b> %{y:.1f}'
        + '<extra></extra>',
))

dfD1 = dfD[dfD['CR'].between(1, 20)]

fig.add_trace(go.Scatter(
    x=dfD1['CR'], 
    y=dfD1['Legendary Resistance eHP'],
    mode='lines+markers', 
    unselected=dict(marker=dict(opacity=0.0)),
    name='monster legendary resistance effective HP',
    hovertemplate = '<b>Monster CR</b> %{x}<br>'
        + '<b>eHP</b> %{y:.1f}'
        + '<extra></extra>',
))

# show figure
fig.update_layout(width=600, height=500)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name=f'fig-pcs-encounter-edpr-legendary-resistance-vs-level-{encounter_difficulty.lower()}-adventuring-days-large')
    tfb.save_fig_html(fig, format='small', name=f'fig-pcs-encounter-edpr-legendary-resistance-vs-level-{encounter_difficulty.lower()}-adventuring-days-small')